In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [2]:
import torch
from cifar_dataset import CIFAR100
from utils.resnet_model import ResNet18, ResNet50, ResNet34
import utils.skorch_trainer as skorch_trainer
from torchvision.transforms import v2
from utils.skorch_trainer import DataAug

train_data = CIFAR100(split='train').dataset
test_set = CIFAR100(split='test').dataset

train_size = int(0.8 * len(train_data))
test_size = len(train_data) - train_size

train_set, val_set = torch.utils.data.random_split(
    CIFAR100(split='train').dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

len(train_set), len(val_set), len(test_set), train_set[0][0].shape

(40000, 10000, 10000, torch.Size([3, 32, 32]))

In [3]:
epochs = 3
lr = 0.1

net_18_pure = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_pure',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = lr,
        module__output_features = 100,
        train_split = val_set,
        callbacks = []
    )
)

net_18_pure = skorch_trainer.net_fit(
    net_18_pure, train_set, None, epochs)



 New training:


: 

In [ ]:
max_epochs = 40
lr = 0.01

t1 = v2.Compose([
    v2.RandomChoice([
        v2.RandomPerspective(distortion_scale=0.1, p=0.8), 
        v2.RandomRotation(degrees=(0, 360))])
])

train_set_t1, _ = torch.utils.data.random_split(
    CIFAR100(split='train', transform=t1).dataset, [train_size, test_size], 
    generator=torch.Generator().manual_seed(42))

net_18_t1 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_t2',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = lr,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        batch_size = 1024,
        callbacks = []
    )
)

net_18_t1 = skorch_trainer.net_fit(
    net_18_t1, train_set_t1, None, max_epochs)

In [10]:
net_18_test_1 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_test_1',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = 0.1,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        callbacks = [
            DataAug(number_samples=5, p=0.75),
        ]
    )
)

net_18_test_1 = skorch_trainer.net_fit(
    net_18_test_1, train_set, None, 50, new_lr=0.01)

Load saved params: 18_test_1_params.pt
Histories:
  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      lr      dur
-------  ------------  -----------  ----------  ------------  ----  ------  -------
      1        4.2010       0.0490      0.0458        4.4809     +  0.1000  70.3854
      2        3.4543       0.1590      0.1416        3.5411     +  0.1000  67.5431
      3        3.0922       0.1729      0.1614        3.6213     +  0.1000  67.0053
      4        2.7750       0.2928      0.2918        2.9550     +  0.1000  69.1558
      5        2.4832       0.2992      0.2881        3.0181        0.1000  68.5017
      6        2.3963       0.2600      0.2579        3.0554        0.1000  67.2076
      7        2.3404       0.2881      0.2844        2.8913        0.1000  69.1046
      8        2.1058       0.3575      0.3492        2.7137     +  0.1000  67.1878
      9        2.0162       0.2710      0.2737        3.0702        0.1000  68.5860
     10        1.9524     

In [21]:
net_18_test_2 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_test_2',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = 0.1,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        callbacks = [
            DataAug(number_samples=10, th=0.1, p=1),
        ]
    )
)

net_18_test_2 = skorch_trainer.net_fit(
    net_18_test_2, train_set, None, 60, new_lr=0.001, load_best=True)

Load saved params: 18_test_2_params.pt
Histories:
  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      lr       dur
-------  ------------  -----------  ----------  ------------  ----  ------  --------
      1        4.3590       0.0883      0.0597        3.9389     +  0.1000  139.3596
      2        3.7472       0.1604      0.1328        3.5306     +  0.1000  140.4540
      3        3.3872       0.2000      0.1749        3.3332     +  0.1000  140.8567
      4        3.1448       0.2437      0.2157        3.1264     +  0.1000  136.6089
      5        2.8625       0.3017      0.2913        2.8108     +  0.1000  138.4741
      6        2.6110       0.3489      0.3386        2.6117     +  0.1000  137.8720
      7        2.3826       0.3931      0.3856        2.4196     +  0.1000  139.0012
      8        2.1503       0.4155      0.4065        2.3264     +  0.1000  139.4589
      9        1.9665       0.4196      0.4153        2.2877     +  0.1000  139.7233
     10        

In [24]:
net_18_test_3 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_test_3',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = 0.1,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        callbacks = [
            DataAug(number_samples=3, th=0.1, p=1),
        ]
    )
)

net_18_test_3 = skorch_trainer.net_fit(
    net_18_test_3, train_set, None, 40, new_lr = 0.001, load_best=True)

Load saved params: 18_test_3_params.pt
Histories:
  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      lr      dur
-------  ------------  -----------  ----------  ------------  ----  ------  -------
      1        4.2574       0.1202      0.0877        3.7375     +  0.1000  63.8545
      2        3.5982       0.1805      0.1558        3.3773     +  0.1000  64.5239
      3        3.3002       0.2034      0.1892        3.2434     +  0.1000  64.6936
      4        3.0555       0.2679      0.2511        2.9436     +  0.1000  64.5847
      5        2.8417       0.2724      0.2636        2.9695     +  0.1000  64.6195
      6        2.7171       0.3106      0.3003        2.7352     +  0.1000  64.7121
      7        2.5489       0.3243      0.3121        2.6945     +  0.1000  64.4163
      8        2.3856       0.3538      0.3470        2.5926     +  0.1000  64.5267
      9        2.3085       0.3621      0.3505        2.5601     +  0.1000  64.5878
     10        2.1119     

In [31]:
net_18_test_4 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_test_4',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = 0.1,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        callbacks = [
            DataAug(number_samples=5, th=0.1, p=0.75),
        ]
    )
)

net_18_test_4 = skorch_trainer.net_fit(
    net_18_test_4, train_set, None, 50, new_lr=0.01, load_best=True)

Load saved params: 18_test_4_params.pt
Histories:
  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      lr      dur
-------  ------------  -----------  ----------  ------------  ----  ------  -------
      1        4.1642       0.0732      0.0594        4.1459     +  0.1000  70.0652
      2        3.4982       0.1213      0.1185        3.7710     +  0.1000  70.2317
      3        3.0407       0.1773      0.1754        3.3905     +  0.1000  70.0554
      4        2.6815       0.2342      0.2297        3.1293     +  0.1000  68.8142
      5        2.4300       0.2715      0.2697        2.9549     +  0.1000  69.4995
      6        2.2872       0.3486      0.3451        2.6103     +  0.1000  69.6286
      7        2.0931       0.3459      0.3494        2.6252     +  0.1000  69.2546
      8        1.9390       0.3440      0.3394        2.6690        0.1000  69.0941
      9        1.8973       0.3940      0.3927        2.4129     +  0.1000  69.3488
     10        1.7598     

In [37]:
net_18_test_5 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_test_5',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = 0.1,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        callbacks = [
            DataAug(number_samples=5, th=0.1, p=1),
        ]
    )
)

net_18_test_5 = skorch_trainer.net_fit(
    net_18_test_5, train_set, None, 50, new_lr= 0.001, load_best=True)

Load saved params: 18_test_5_params.pt
Histories:
  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      lr      dur
-------  ------------  -----------  ----------  ------------  ----  ------  -------
      1        4.1626       0.1244      0.0995        3.6623     +  0.1000  81.3333
      2        3.5230       0.1760      0.1510        3.4180     +  0.1000  81.8533
      3        3.2189       0.2637      0.2442        2.9290     +  0.1000  81.9834
      4        2.9106       0.2623      0.2497        3.0437     +  0.1000  81.6420
      5        2.7955       0.3306      0.3161        2.6727     +  0.1000  80.9916
      6        2.5583       0.3402      0.3248        2.6241     +  0.1000  81.6839
      7        2.3569       0.3579      0.3484        2.5627     +  0.1000  82.7941
      8        2.1757       0.3824      0.3724        2.4473     +  0.1000  82.7396
      9        2.0690       0.3949      0.3884        2.4091     +  0.1000  80.1853
     10        1.9387     

In [41]:
net_18_test_6 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_test_6',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = 0.1,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        callbacks = [
            DataAug(number_samples=5, th=0.3, p=1),
        ]
    )
)

net_18_test_6 = skorch_trainer.net_fit(
    net_18_test_6, train_set, None, 35, new_lr=0.001, load_best=True)

Load saved params: 18_test_6_params.pt
Histories:
  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      lr      dur
-------  ------------  -----------  ----------  ------------  ----  ------  -------
      1        4.2905       0.1149      0.0861        3.7698     +  0.1000  80.2533
      2        3.6638       0.1551      0.1288        3.5864     +  0.1000  79.7673
      3        3.4095       0.2028      0.1843        3.3103     +  0.1000  79.7210
      4        3.1715       0.2451      0.2339        3.1382     +  0.1000  79.8440
      5        2.9172       0.2973      0.2869        2.9308     +  0.1000  79.9136
      6        2.7053       0.3202      0.3048        2.7669     +  0.1000  82.9438
      7        2.5488       0.3676      0.3534        2.5203     +  0.1000  84.3551
      8        2.3879       0.3564      0.3495        2.5896        0.1000  82.3732
      9        2.4110       0.3278      0.3179        2.7236        0.1000  83.2572
     10        2.2514     

In [42]:
net_18_test_7 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_test_7',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = 0.1,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        callbacks = [
            DataAug(number_samples=5, th=0.7, p=1),
        ]
    )
)

net_18_test_7 = skorch_trainer.net_fit(
    net_18_test_7, train_set, None, 20, load_best=True)



 New training:
  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      lr      dur
-------  ------------  -----------  ----------  ------------  ----  ------  -------
      1        4.3031       0.1034      0.0775        3.8544     +  0.1000  81.8084
      2        3.6852       0.1610      0.1350        3.5238     +  0.1000  80.9431
      3        3.3848       0.2112      0.1935        3.2458     +  0.1000  81.6473
      4        3.1653       0.2077      0.1961        3.3087     +  0.1000  80.7408
      5        2.9628       0.2930      0.2749        2.8169     +  0.1000  79.9978
      6        2.6678       0.3073      0.2960        2.7817     +  0.1000  81.7626
      7        2.5565       0.2950      0.2851        2.8692        0.1000  80.1296
      8        2.5219       0.3174      0.3060        2.7935     +  0.1000  79.9359
      9        2.3710       0.3347      0.3202        2.6917     +  0.1000  79.8024
     10        2.2363       0.3661      0.3552        2.576

In [12]:
net_18_t8 = skorch_trainer.net_def(
    ResNet18, 
    net_name = '18_test_8',
    classes=[torch.tensor, torch.tensor],    
    classifier_kwargs = dict(
        lr = 0.1,
        module__output_features = 100,
        module__weights = None,
        train_split = val_set,
        callbacks = [
            DataAug(number_samples=8, th=0.1, p=1),
        ]
    )
)

net_18_t8 = skorch_trainer.net_fit(
    net_18_t8, train_set, None, 25, new_lr=0.01, load_best=True)

Load saved params: 18_test_8_params.pt
Histories:
  epoch    train_loss    valid_acc    valid_f1    valid_loss    cp      lr       dur
-------  ------------  -----------  ----------  ------------  ----  ------  --------
      1        4.2135       0.1183      0.0925        3.8846     +  0.1000  114.5282
      2        3.5326       0.1878      0.1661        3.3720     +  0.1000  117.3779
      3        3.2599       0.2114      0.1912        3.2648     +  0.1000  119.5108
      4        2.9836       0.2817      0.2635        2.8676     +  0.1000  115.6632
      5        2.7255       0.3414      0.3276        2.6326     +  0.1000  119.5449
      6        2.5465       0.3010      0.2894        2.8241        0.1000  116.6128
      7        2.2796       0.3921      0.3857        2.4194     +  0.1000  115.6344
      8        2.1684       0.3809      0.3696        2.4015        0.1000  122.2044
      9        1.9799       0.4374      0.4314        2.1586     +  0.1000  124.1382
     10        